## How to use nn module

In [45]:
import torch
import torch.nn as nn

class Model(nn.Module):

    def __init__(self, num_features):
        super().__init__()
        # Hidden Layer containing 3 neurons with relu activation  -->  input shape = num_features, output shape = 3
        # self.linear1 = nn.Linear(num_features, 3)
        # self.relu = nn.ReLU()

        # Output Layer (input = output of hidden layer)
        # self.linear2 = nn.Linear(3, 1)
        # self.sigmoid = nn.Sigmoid()

        # using Sequential
        self.network = nn.Sequential(
            nn.Linear(num_features, 3),
            nn.ReLU(),
            nn.Linear(3, 1),
            nn.Sigmoid()
        )

    def forward(self, features):
        # out = self.linear1(features)
        # out = self.relu(out)
        # out = self.linear2(out)
        # out = self.sigmoid(out)

        out = self.network(features)

        return out

In [46]:
# create dataset
features = torch.rand(10, 5)

# create model
model = Model(features.shape[1])

# call model for forward pass
model(features)                 # magic method in the class calls the forward method when the object is made
# model.forward(features)        # works but not recommended

tensor([[0.3951],
        [0.3906],
        [0.3951],
        [0.3929],
        [0.3906],
        [0.3943],
        [0.3905],
        [0.3950],
        [0.3908],
        [0.3951]], grad_fn=<SigmoidBackward0>)

In [ ]:
model.linear1.weight

In [33]:
!pip install torchinfo        # for model visualization

In [48]:
from torchinfo import summary
summary(model, input_size=(10, 5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Sequential: 1-1                        [10, 1]                   --
│    └─Linear: 2-1                       [10, 3]                   18
│    └─ReLU: 2-2                         [10, 3]                   --
│    └─Linear: 2-3                       [10, 1]                   4
│    └─Sigmoid: 2-4                      [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

## Trainig pipeline using nn module

In [49]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [50]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [51]:
df.shape

(569, 33)

In [52]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [53]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


### train test split

In [54]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

### scaling

In [55]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [56]:
X_train

array([[ 1.17770169e-01,  1.10647536e-01,  2.04728725e-01, ...,
        -1.83869877e-01, -9.93836795e-01,  1.85404709e-01],
       [-6.62927555e-01,  1.17867385e+00, -6.14235614e-01, ...,
         2.45710042e-01, -1.70797549e-01,  8.29031970e-01],
       [-6.31355221e-01, -4.49442696e-01, -6.60474190e-01, ...,
        -9.10413755e-01, -2.07997063e-01, -1.87250529e-01],
       ...,
       [-1.09058918e+00, -3.99556116e-01, -1.08578578e+00, ...,
        -1.17759151e+00, -7.00488650e-02, -3.58737895e-01],
       [ 1.10512317e+00, -1.36327413e+00,  9.78287615e-01, ...,
        -4.63620703e-01, -1.75177689e+00, -1.42393826e+00],
       [-2.86106539e-02, -4.63481706e-04, -9.93628147e-02, ...,
        -4.92209471e-01, -8.63638497e-01, -4.71963528e-01]])

In [57]:
y_train

290    B
537    B
399    B
390    B
404    B
      ..
54     M
195    B
217    B
491    B
481    B
Name: diagnosis, Length: 455, dtype: object

### Label Encoding

In [58]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [59]:
y_train

array([0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,

### Numpy arrays to PyTorch tensors

In [60]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [61]:
X_train_tensor.shape

torch.Size([455, 30])

In [ ]:
y_train_tensor.shape

torch.Size([455])

### Defining the model

In [62]:
import torch.nn as nn


class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):

    out = self.linear(features)
    out = self.sigmoid(out)

    return out

### Important Parameters

In [63]:
learning_rate = 0.1
epochs = 25

In [64]:
# define loss function
loss_function = nn.BCELoss()

### Training Pipeline

In [68]:
# create model
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model(X_train_tensor)

  # loss calculate
  loss = loss_function(y_pred, y_train_tensor.view(-1,1))          # view == reshape

  # clear gradients
  optimizer.zero_grad()

  # backward pass
  loss.backward()

  # parameters update
  optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.7093659043312073
Epoch: 2, Loss: 0.5327824950218201
Epoch: 3, Loss: 0.43983808159828186
Epoch: 4, Loss: 0.3832872211933136
Epoch: 5, Loss: 0.3448967933654785
Epoch: 6, Loss: 0.31682685017585754
Epoch: 7, Loss: 0.2952043116092682
Epoch: 8, Loss: 0.27789878845214844
Epoch: 9, Loss: 0.26364001631736755
Epoch: 10, Loss: 0.2516217827796936
Epoch: 11, Loss: 0.24130648374557495
Epoch: 12, Loss: 0.23232108354568481
Epoch: 13, Loss: 0.22439786791801453
Epoch: 14, Loss: 0.21733935177326202
Epoch: 15, Loss: 0.2109963744878769
Epoch: 16, Loss: 0.20525376498699188
Epoch: 17, Loss: 0.20002111792564392
Epoch: 18, Loss: 0.19522631168365479
Epoch: 19, Loss: 0.19081099331378937
Epoch: 20, Loss: 0.18672727048397064
Epoch: 21, Loss: 0.18293549120426178
Epoch: 22, Loss: 0.1794024556875229
Epoch: 23, Loss: 0.17610011994838715
Epoch: 24, Loss: 0.17300458252429962
Epoch: 25, Loss: 0.1700952649116516


### Evaluation

In [69]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')


Accuracy: 0.5123114585876465
